In [1]:
from __future__ import print_function
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential, load_model, model_from_json
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Convolution2D, MaxPooling2D, Flatten, MaxPooling2D, Lambda
import pickle
import csv
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json


Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [2]:
data_dir= "data/"
data_file = "driving_log.csv"

image_file_list=[]
center_image_data_list=[]
steering_angles_list=[]

image_y = 24
image_x = 64

image_y_orig=160
image_x_orig=320 

min_angle_threshold=0.01
max_angle_threshold=.5


with open(data_dir+data_file, mode='rt') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader, None)
    for row in reader:
#         #print(row[3])
#         #print(row[0])
#         image=cv2.imread(row[0])
#         print("Original image is shape:", image.shape)
#         cropped=image[round(0.25*image_y_orig):(image_y_orig),0:(image_x_orig),:]
#         print("Cropped image is shape:", cropped.shape)
#         cv2.imshow('original image',image)
#         cv2.waitKey() 
#         cv2.imshow('cropped  image',cropped)
#         cv2.waitKey() 
        
#         image_resized=cv2.resize(cv2.imread(row[0])[round(0.25*image_y_orig):(image_y_orig),0:(image_x_orig),:],(image_x,image_y))
#         #print("Resized image is shape:", image_resized.shape)
#         cv2.imshow('cropped and resized 1',image_resized)
#         cv2.waitKey()
#         #cv2.destroyAllWindows()
#         print(data_dir+row[0])
        angle = float(row[3])
        
        if abs(angle)>=min_angle_threshold and abs(angle)<=max_angle_threshold:
            center_image_data_list.append(cv2.resize(cv2.imread(data_dir+row[0])[round(0.25*image_y_orig):(image_y_orig),0:(image_x_orig),:],(image_x,image_y)))
            steering_angles_list.append(float(row[3]))      
        
            center_image_data_list.append(cv2.flip(cv2.resize(cv2.imread(data_dir+row[0])
                                                 [round(0.25*image_y_orig):(image_y_orig),0:(image_x_orig),:],(image_x,image_y)),1))
            steering_angles_list.append(float(row[3])*(-1.0))
        
# cv2.imshow('cropped and resized',center_image_data_list[0])   
# cv2.waitKey()
# cv2.imshow('cropped and resized and flipped',center_image_data_list[1])   
# cv2.waitKey()
# cv2.destroyAllWindows()
# cv2.imwrite('lobes.jpg', center_image_data_list[0]) 
# cv2.imwrite('lobes2.jpg', center_image_data_list[1]) 
        
        
center_image_data=np.asarray(center_image_data_list)
steering_angles=np.asarray(steering_angles_list)     

input_shape=center_image_data[0].shape
print(center_image_data.shape)
print(steering_angles.shape)
print(center_image_data[0].shape)
print(input_shape)

#cv2.imshow('image',center_image_data[0])
#cv2.waitKey()    



(7080, 24, 64, 3)
(7080,)
(24, 64, 3)
(24, 64, 3)


In [ ]:
cv2.imshow('image',center_image_data[0])
cv2.waitKey() 

In [ ]:
import matplotlib.pyplot as plt

testimage=cv2.cvtColor(center_image_data[0], cv2.COLOR_BGR2RGB)

testimageback=cv2.cvtColor(testimage, cv2.COLOR_RGB2BGR)

plt.imshow(testimage) 
plt.show()

print(testimage.shape)

In [ ]:
cv2.imshow('image',testimageback)
cv2.waitKey() 

In [ ]:
print(steering_angles[1000:1005])

In [ ]:
model_savepath = "CompleteModel.h5"
model_weights_savepath = "model.h5"
model_json_savepath = "model.json"

In [ ]:
Conv1_Channels = 24
Conv1_Pixels=4
Conv1_Stride=2
Conv2_Channels = 32
Conv2_Pixels=4
Conv2_Stride=2
Conv3_Channels = 32
Conv3_Pixels=3
Conv3_Stride=2
fc1_neurons=100
fc2_neurons=25
fc3_neurons=5
fc4_neurons=1


model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=input_shape))
model.add(Convolution2D(Conv1_Channels, Conv1_Pixels, Conv1_Pixels,
                        border_mode='valid', subsample=(Conv1_Stride,Conv1_Stride)))
model.add(Activation('relu'))
model.add(Convolution2D(Conv2_Channels, Conv2_Pixels, Conv2_Pixels,
                        border_mode='valid', subsample=(Conv2_Stride,Conv2_Stride)))
model.add(Activation('relu'))
model.add(Convolution2D(Conv3_Channels, Conv3_Pixels, Conv3_Pixels,
                        border_mode='valid', subsample=(Conv3_Stride,Conv3_Stride)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Flatten())
# model.add(Dense(fc1_neurons))
# model.add(Activation('relu'))
model.add(Dense(fc2_neurons))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(fc3_neurons))
model.add(Dense(fc4_neurons))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer="adam", metrics=['accuracy'])



In [ ]:
batch_size = 100
nb_epoch = 5

history = model.fit(center_image_data, steering_angles,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1)

#history = model.fit_generator(generator, samples_per_epoch, nb_epoch=nb_epoch, verbose=1)


#del model  # deletes the existing model


In [ ]:
score = model.evaluate(center_image_data, steering_angles, verbose=0)
prediction = model.predict(center_image_data, verbose=0)

print("Model loss is:", score[0])
print("Model predictions are:", prediction)
print("Input steering angles are:", steering_angles)
# print('Test score:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
model.save(model_savepath)  # creates a HDF5 file 'my_model.h5'
model.save_weights(model_weights_savepath)
json_string = model.to_json()
with open(model_json_savepath, 'w') as outfile:
    json.dump(json_string, outfile)

In [ ]:
model = load_model(model_savepath)

batch_size = 100
nb_epoch = 1

history = model.fit(center_image_data, steering_angles,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1)




In [ ]:
score = model.evaluate(center_image_data, steering_angles, verbose=0)
prediction = model.predict(center_image_data, verbose=0)

print("Model loss is:", score[0])
print("Model predictions are:", prediction)
print("Input steering angles are:", steering_angles)
# print('Test score:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
model.save(model_savepath)  # creates a HDF5 file 'my_model.h5'
model.save_weights(model_weights_savepath)
json_string = model.to_json()
with open(model_json_savepath, 'w') as outfile:
    json.dump(json_string, outfile)


In [ ]:
def batch_generator(driving_log, batch_size=32):
    num_rows = <figure out total number of rows in csv>
    train_images = np.zeros((batch_size, img_rows, img_cols, 3))
    train_steering = np.zeros(batch_size)
    ctr = None
    while 1:        
        for j in range(batch_size):
            if ctr is None or ctr >= num_rows:
                ctr = 0     # Initialize counter or reset counter if over bounds
            train_images[j], train_steering[j] = <load data from row number "ctr" in the CSV file> 
            ctr += 1
        yield train_images, train_steering

In [ ]:
# Nvidia Model

# Conv1_Channels = 24
# Conv1_Pixels=5
# Conv1_Stride=2
# Conv2_Channels = 36
# Conv2_Pixels=5
# Conv2_Stride=2
# Conv3_Channels = 48
# Conv3_Pixels=5
# Conv3_Stride=2
# Conv4_Channels = 64
# Conv4_Pixels=3
# Conv4_Stride=2
# Conv5_Channels = 64
# Conv5_Pixels=3
# Conv5_Stride=1
# fc1_neurons=100
# fc2_neurons=50
# fc3_neurons=10
# fc4_neurons=1

# model = Sequential()
# model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=input_shape))
# model.add(Convolution2D(Conv1_Channels, Conv1_Pixels, Conv1_Pixels,
#                         border_mode='valid', subsample=(Conv1_Stride,Conv1_Stride)))
# model.add(Activation('relu'))
# model.add(Convolution2D(Conv2_Channels, Conv2_Pixels, Conv2_Pixels,
#                         border_mode='valid', subsample=(Conv2_Stride,Conv2_Stride)))
# model.add(Activation('relu'))
# model.add(Convolution2D(Conv3_Channels, Conv3_Pixels, Conv3_Pixels,
#                         border_mode='valid', subsample=(Conv3_Stride,Conv3_Stride)))
# model.add(Activation('relu'))
# model.add(Convolution2D(Conv4_Channels, Conv4_Pixels, Conv4_Pixels,
#                         border_mode='valid', subsample=(Conv4_Stride,Conv4_Stride)))
# model.add(Activation('relu'))
# model.add(Convolution2D(Conv5_Channels, Conv5_Pixels, Conv5_Pixels,
#                         border_mode='valid', subsample=(Conv5_Stride,Conv5_Stride)))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(fc1_neurons))
# model.add(Activation('relu'))
# model.add(Dense(fc2_neurons))
# model.add(Activation('relu'))
# model.add(Dense(fc3_neurons))
# model.add(Dense(fc4_neurons))
